# LoFTR demo with custom image pairs on Colab
We provide an easier way to run LoFTR with a custom image pair without configuring a python environment with GPU support. Signing in your Google account is required to run this notebook in Colab.

Start by clicking Runtime --> Run all (Ctrl/Cmd + F9). 

Upload your own image pair with the "Choose Files" button to appear in the first cell. Please use horizontal images (width > height) and assign the image type (indoor/outdoor) accordingly.

In [ ]:
# print("Please wait for ~8 seconds for the GPU session initialization. \n \
# ==> Please select both images at the same time after clicking \"Choose Files\".")
# !mkdir -p /content/uploaded/ && rm -rf /content/uploaded/* # clear previously uploaded images
# %cd /content/uploaded/
# from google.colab import files
# uploaded = files.upload()

# for file_name in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=file_name, length=len(uploaded[file_name])))
# image_pair = ['/content/uploaded/' + f for f in list(uploaded.keys())]
# %cd ..

# # Change the image type here.
# image_type = 'indoor'
# image_type = 'outdoor'

You can also choose to use the example image pair provided in the LoFTR repo (from ScanNet) by using this cell (uncommenting the last line).


In [ ]:
# img0_pth = "uploaded/0aafg-missing.jpg"
# img1_pth = "uploaded/0aafg.jpg"
# image_pair = [img0_pth, img1_pth]
# image_pair = [img1_pth, img0_pth]

In [ ]:
# # Configure environment and grab LoFTR code.
# # !rm -rf sample_data
# # !pip install einops yacs kornia
# # !git clone https://github.com/zju3dv/LoFTR --depth 1
# # !mv LoFTR/* . && rm -rf LoFTR

# # Download pretrained weights
# !mkdir weights 
# %cd weights/
# !gdown --id 1w1Qhea3WLRMS81Vod_k5rxS_GNRgIi-O  # indoor-ds
# !gdown --id 1M-VD35-qdB5Iw-AtbDBCKC7hPolFW9UY  # outdoor-ds
# %cd ..

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.cm as cm
from tqdm import tqdm
from src.utils.plotting import make_matching_figure
from src.loftr import LoFTR, default_cfg

In [ ]:
# #!pip install -qq aicrowd-cli
# %load_ext aicrowd.magic

In [ ]:
# %aicrowd login

In [ ]:
# %aicrowd ds dl -c face-recognition -o data

In [ ]:
# !unzip data/data.zip -d /content/data > /dev/null

In [ ]:
import os
image_ids = os.listdir("data/missing")
len(image_ids)

In [ ]:
def get_target_face(face_no, target_image):
  ''' This function helps to retrieve the individual faces from the main patch of 100 images'''

  x, y = (int(face_no[0]))*216, (int(face_no[1]))*216

  target_face = target_image[x:x+216, y:y+216]

  return target_face

In [ ]:
# default_cfg['match_coarse']['match_type'] = 'sinkhorn'
default_cfg['match_coarse']['thr'] = 0.05

for k in default_cfg['match_coarse'].keys():
    print(k)
    print(k, default_cfg['match_coarse'][k])
    
default_cfg

In [ ]:
# The default config uses dual-softmax.
# The outdoor and indoor models share the same config.
# You can change the default values like thr and coarse_match_type.
matcher = LoFTR(config=default_cfg)

matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.eval().cuda()

In [ ]:
from superglue import log_optimal_transport


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!ls -lthr


In [ ]:
predictions = {"ImageID":[], "target":[]}

all_results=[]
all_inliers=[]

for img_id in tqdm(image_ids):
    missing_image = cv2.imread(os.path.join("data/missing", img_id))[100:412,100:412,:]
    missing_image = cv2.resize(missing_image, (640, 480))
    missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGR2GRAY) #convert to gray scale
    missing = torch.from_numpy(missing_image)[None][None].cuda() / 255.

    target_image = cv2.imread(os.path.join("data/target", img_id))

    matches = []
    inliers = []

    for face_no in range(100):
        face_no = str(face_no)
        face_no = face_no.zfill(2)

        target_face = get_target_face(face_no, target_image)
        target_face = cv2.cvtColor(target_face, cv2.COLOR_BGR2GRAY)
        target_face = cv2.resize(target_face,(640,480))

        face = torch.from_numpy(target_face)[None][None].cuda() / 255.
        batch = {'image0': missing, 'image1': face}

        # Inference with LoFTR and get prediction
        with torch.no_grad():
            matcher(batch)
            mkpts0 = batch['mkpts0_f'].cpu().numpy()
            mkpts1 = batch['mkpts1_f'].cpu().numpy()
            mconf = batch['mconf'].cpu().numpy()

        src_pts = np.float32(mkpts0).reshape(-1,1,2)
        dst_pts = np.float32(mkpts1).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        inliers_cnt = sum(matchesMask)
        percent = inliers_cnt/len(matchesMask)
        
        matches.append(len(mkpts0))
        inliers.append(percent)
    all_results.append(matches)
    all_inliers.append(inliers)
    best_id = int(np.argmax(np.array(inliers)*np.array(matches)))
    print(img_id,best_id)
    predictions['ImageID'].append(img_id.replace(".jpg", ""))
    predictions['target'].append(best_id)

    # target_face = get_target_face('%02d'%best_id, target_image)
    # target_face = cv2.cvtColor(target_face, cv2.COLOR_BGR2GRAY)
    # target_face = cv2.resize(target_face,(640,480))

    # face = torch.from_numpy(target_face)[None][None].cuda() / 255.
    # batch = {'image0': missing, 'image1': face}

    # # Inference with LoFTR and get prediction
    # with torch.no_grad():
    #     matcher(batch)
    #     mkpts0 = batch['mkpts0_f'].cpu().numpy()
    #     mkpts1 = batch['mkpts1_f'].cpu().numpy()
    #     mconf = batch['mconf'].cpu().numpy()
    # # Draw 
    # color = cm.jet(mconf, alpha=0.7)
    # text = [
    #     'LoFTR',
    #     'Matches: {}'.format(len(mkpts0)),
    # ]
    # fig = make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text)

    # # A high-res PDF will also be downloaded automatically.
    # make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text, path="LoFTR-colab-demo.pdf")
    # # files.download("LoFTR-colab-demo.pdf")


In [ ]:
plt.subplot(2,1,1)
plt.plot(all_results[0],label='matches');
plt.legend()
plt.subplot(2,1,2)
plt.plot(np.array(all_inliers[0]),label='percent inliers');
plt.legend()

In [ ]:
ii=1
plt.subplot(2,1,1)
plt.plot(all_results[ii],label='matches');
plt.legend()
plt.subplot(2,1,2)
plt.plot(np.array(all_inliers[ii]),label='percent inliers');
plt.legend()

In [ ]:
all_results = np.array(all_results)
all_inliers = np.array(all_inliers)
all_results.dtype

In [ ]:
all_results,all_inliers,all_results*all_inliers

In [ ]:
plt.plot(all_results.transpose());

In [ ]:
plt.plot(all_inliers.transpose());

In [ ]:
plt.figure(figsize=(20,5))
srtd = np.sort(all_results, axis=1)
plt.imshow(srtd[:,-10:])
plt.title('matches')
print(srtd[:,-1]-srtd[:,-2])

plt.figure(figsize=(20,5))
srtd = np.sort(all_inliers, axis=1)
plt.imshow(srtd[:,-10:])
print(srtd[:,-1]-srtd[:,-2])
plt.title('inliers')


In [ ]:
# best3 = all_results.argsort(axis=1)[:,-2:]
# best3 = all_inliers.argsort(axis=1)[:,-2:]
best3 = (all_results*all_inliers).argsort(axis=1)[:,-2:]



In [ ]:
for img_id,b3,res in list(zip(image_ids,best3,all_results)):
    plt.figure(figsize=(20,20))
    plt.subplot(3,2,1)
    plt.plot(res)
    print(img_id,b3)
    missing_image = cv2.imread(os.path.join("data/missing", img_id))[100:412,100:412,:]
    print(missing_image.shape)
    missing_image = cv2.resize(missing_image, (640, 480))
#     missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGR2GRAY) #convert to gray scale
#     missing = torch.from_numpy(missing_image)[None][None].cuda() / 255.

    target_image = cv2.imread(os.path.join("data/target", img_id))
    for iii, best_id in enumerate(b3):
        target_face = get_target_face('%02d'%best_id, target_image)
#         target_face = cv2.cvtColor(target_face, cv2.COLOR_BGR2GRAY)
        target_face = cv2.resize(target_face,(640,480))

#         face = torch.from_numpy(target_face)[None][None].cuda() / 255.
#         batch = {'image0': missing, 'image1': face}

#         # Inference with LoFTR and get prediction
#         with torch.no_grad():
#             matcher(batch)
#             mkpts0 = batch['mkpts0_f'].cpu().numpy()
#             mkpts1 = batch['mkpts1_f'].cpu().numpy()
#             mconf = batch['mconf'].cpu().numpy()
#         # Draw 
#         color = cm.jet(mconf, alpha=0.7)
#         text = [
#             'LoFTR',
#             'Matches: {}'.format(len(mkpts0)),
#         ]
        plt.subplot(3,2,3+2*iii)
        plt.imshow(missing_image[:,:,::-1])
        plt.subplot(3,2,4+2*iii)
        plt.imshow(target_face[:,:,::-1])
#     fig = make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text)
        
        # A high-res PDF will also be downloaded automatically.
#         make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text)#, path="LoFTR-colab-demo.pdf")
        # files.download("LoFTR-colab-demo.pdf")


In [ ]:

target_face = get_target_face('%02d'%best_id, target_image)
target_face = cv2.cvtColor(target_face, cv2.COLOR_BGR2GRAY)
target_face = cv2.resize(target_face,(640,480))

face = torch.from_numpy(target_face)[None][None].cuda() / 255.
batch = {'image0': missing, 'image1': face}

# Inference with LoFTR and get prediction
with torch.no_grad():
    matcher(batch)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()
# Draw 
color = cm.jet(mconf, alpha=0.7)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text)

# A high-res PDF will also be downloaded automatically.
make_matching_figure(missing_image, target_face, mkpts0, mkpts1, color, mkpts0, mkpts1, text, path="LoFTR-colab-demo.pdf")
# files.download("LoFTR-colab-demo.pdf")


In [ ]:
mkpts0.shape

In [ ]:

src_pts = np.float32(mkpts0).reshape(-1,1,2)
dst_pts = np.float32(mkpts1).reshape(-1,1,2)
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()
inliers = sum(matchesMask)
percent = inliers/len(matchesMask)
h,w,_ = missing_image.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)
img2 = cv2.polylines(target_face.copy(),[np.int32(dst)],True,255,3, cv2.LINE_AA)

plt.imshow(img1[:,:,::-1])
plt.figure()
plt.imshow(img2)

In [ ]:
sum(matchesMask),percent

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
MIN_MATCH_COUNT = 10
img1 = missing_image.copy()
img2 = target_face.copy()
# Initiate SIFT detector
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
#     if m.distance < 0.7*n.distance:
    good.append(m)

In [ ]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w,_ = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
plt.imshow(img3, 'gray'),plt.show()

In [ ]:
!ls weights

In [ ]:
import pandas as pd

In [ ]:
submission = pd.DataFrame(predictions)
submission.head()

In [ ]:
!rm -rf assets
!mkdir assets
submission.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [ ]:
%aicrowd notebook submit -c face-recognition -a assets --no-verify

In [ ]:
# # Rerun this cell (and below) if a new image pair is uploaded.
# img0_raw = cv2.imread(image_pair[0], cv2.IMREAD_GRAYSCALE)
# img1_raw = cv2.imread(image_pair[1], cv2.IMREAD_GRAYSCALE)
# img0_raw = cv2.resize(img0_raw, (640, 480))
# img1_raw = cv2.resize(img1_raw, (640, 480))

# img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
# img1 = torch.from_numpy(img1_raw)[None][None].cuda() / 255.
# batch = {'image0': img0, 'image1': img1}

# # Inference with LoFTR and get prediction
# with torch.no_grad():
#     matcher(batch)
#     mkpts0 = batch['mkpts0_f'].cpu().numpy()
#     mkpts1 = batch['mkpts1_f'].cpu().numpy()
#     mconf = batch['mconf'].cpu().numpy()

In [ ]:
# # Draw 
# color = cm.jet(mconf, alpha=0.7)
# text = [
#     'LoFTR',
#     'Matches: {}'.format(len(mkpts0)),
# ]
# fig = make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, mkpts0, mkpts1, text)

# # A high-res PDF will also be downloaded automatically.
# make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, mkpts0, mkpts1, text, path="LoFTR-colab-demo.pdf")
# files.download("LoFTR-colab-demo.pdf")

In [ ]:
!pwd
